# Exercise 5, Machine Learning 2022

The following lab-session is adapted from those of Sections 5.3 and 6.3 in Introduction to Statistical Learning with R.

### The Auto dataset
We use the `Auto` dataset that was used as an example throughout Chapter 3 on linear regression. Here, we treat the variable `mpg` (gas miles in miles per gallon) as the response and `horsepower` as the single predictor.

The data has 392 observations on the following 9 variables.

`mpg` miles per gallon

`cylinders` Number of cylinders between 4 and 8 displacementEngine displacement (cu. inches) 

`horsepower` Engine horsepower

`weight` Vehicle weight (lbs.)

`acceleration` Time to accelerate from 0 to 60 mph (sec.)

`year` Model year (modulo 100)

`origin` Origin of car (1. American, 2. European, 3. Japanese)

`name` Vehicle name

In [3]:
# Import relevant packages
import numpy as np
from pandas import read_csv, DataFrame
from math import log, sqrt
import matplotlib.pyplot as plt
import random

import statsmodels.api as sm
import statsmodels.formula.api as smf 

In [14]:
# Load data from csv; change the directory as you need!
Auto = read_csv("Auto.csv")
Auto.shape

(392, 9)

*It is recommended that you set a random seed, so that any results based on randomness are recreated whenever you run the notebook.*

In [26]:
## Set seed!
random.seed(10)

## The validation set approach

Split the set of observations into two halves by selecting a random subset of 196 obervations out of the original 392 observations. We refer to these observations as the *training set* and the remaining observations as the *validation set*. For creating the random split, you can use `random.sample(seq, n)` to select `n` numbers without replacement from the sequence `seq`. 

In [27]:
train_idx = random.sample(range(392),196)
train_data = Auto.iloc[train_idx]
val_data = Auto.iloc[[i for i in range(392) if i not in train_idx]]

Fit a linear regression on the training set: `mpg`$= \beta_0 + \beta_1$ `horsepower` $+\epsilon$.

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
292,34.1,4,86.0,65,1975,15.2,79,3,maxda glc deluxe
16,18.0,6,199.0,97,2774,15.5,70,1,amc hornet
219,17.5,8,305.0,145,3880,12.5,77,1,chevrolet caprice classic
247,19.9,8,260.0,110,3365,15.5,78,1,oldsmobile cutlass salon brougham
295,25.4,5,183.0,77,3530,20.1,79,2,mercedes benz 300d
...,...,...,...,...,...,...,...,...,...
231,29.0,4,97.0,78,1940,14.5,77,2,volkswagen rabbit custom
63,15.0,8,318.0,150,4135,13.5,72,1,plymouth fury iii
183,26.0,4,98.0,79,2255,17.7,76,1,dodge colt
74,14.0,8,318.0,150,4077,14.0,72,1,plymouth satellite custom (sw)


Compute the MSE on the 196 observations that are the validation set.

Now estimate also the MSE for the quadradic and cubic regressions.

The ISLwR book found an estimated test MSE of (23.27, 18.27, and 18.79) for the three regression models. 

Set another seed and create a different split of your data - do you get different results? Why?


Which model would you choose based on the estimated MSE? 

If you instead train the models on the entire data and perform model selection via F-tests or AIC, would you choose differently?

## Leave-one-out cross-validation

Implement leave-one-out cross-validation and use it to estimate the MSE for the three regression models above.

## k-fold cross-validation

Estimate the MSE now from k-fold cross-validation that you implement yourself. If you set k=10, then you get Figure 5.6 from the book, but you are welcome to choose another k. Make sure that you properly randomise observations into the k folds! One way is to first make a random permutation of row indices and then chop the re-ordered dataset into k (roughly) even parts.

The LOOCV estimate of the MSE can also be computed automatially with sklearn. In fact, sklearn provides a lot of functionality for cross-validation. However, our objective right now is to understand how the algorithms work, so that you learn to reflect on them and use the automatic functionality appropriately.

Extend your code with a for-loop that iterates over increasing orders of the polynomial (1 to 10). Note that is possible to autogenerate a design matrix with all the terms of the polynomial with functions such as `sklearn.preprocessing.PolynomialFeatures` (It should also work for several features; a degree 2 polynomial in features x1 and x2 would need  $1+x_1+x_2+x_1x_2+x_1^2+x_2^2$, i.e. also the term $x_1x_2$.)

Plot the resulting MSE curve against the degree of the polynomial (this is the complexity or the flexibility). 

Do you get the same conclusion as before regarding your choice of model?

Was it faster than LOOCV?

## Shrinkage/Regularization

Now try the two shrinkage methods we saw on Friday, ridge regression and lasso. To make things a bit more exciting, include in your models all variables available in the dataset, and perhaps a few polynomial terms for `horsepower` now that you know they are relevant. The `name` variable is evidently an identifyer and should not be used as a predictor in the model.

The Lasso estimates for coefficients $\beta$ in a linear regression model are found by minimising $RSS + \lambda\sum_{j=1}^p|\beta_j|$ rather than RSS. 

In Ridge regression, we estimate $\beta$ by minimising $RSS + \lambda\sum_{j=1}^p\beta_j^2$ rather than RSS. 

In both cases, we need to set the regularation parameter $\lambda$. **The parameter $\lambda$ is called `alpha` in the Python implementations**


The relevant method in the statsmodel library is `fit_regularized`, which you use in place of `fit` after instantiating an ols model. If `L1_wt=1`, the fit is the lasso (L1-penalty). Ridge regression (L2-penalty) is obtained by setting `L1_wt=0`.  The reason we need to set parameter `L1_wt` is that the method actually implements the more general 'elastic net' penalty, which is a convex combination of the Lasso penalty and the ridge regression penalty with weight `L1_wt`. The fitting method returns a `RegressionResults` object, from which you can extract `params` and `fittedvalues`, and it has a `predict` method just like the usual linear regression output from ols.

The sklearn libary has methods `Ridge` and `Lasso`. 

**Set aside a test set of about 25% of the data that you can use at the end to estimate the MSE of your final model. Use the remaining data to train the regularised regression models.**

### Ridge regression

First, carry out a ridge regression where the regulation parameter $\lambda$ is fixed at 10. 

In fact, $\lambda = 10$ is a rather arbitrary suggestion. We need to decide on a suitable value for the hyperparameter $\lambda$. This we can do using a dedicated validation set, or we could do it by cross-validation. We do the latter.

Take a suitable range of $\lambda \in [0, \infty)$, for instance you can take a sequence $10^i, i=-2, \ldots, 5$. Using 5-fold cross-validation to estimate the MSE (call it `MSE_lambda`) for each $\lambda$, make a plot of how MSE changes with $\lambda$. Based on this plot, how would you choose $\lambda$? 

Having chosen a $\lambda$, would it be appropriate to report the corresponding `MSE_lambda` as the test error for the ridge regression?
Compute the test MSE using the dedicated test set and compare to `MSE_lambda`.

In fact, once we have chosen the regularisation parameter $\lambda$, we should preferably make use of *all of the data* to (re-)train the model, and then report the associated test MSE from the test data. Do this, and inspect the model estimates.

### Lasso

Following the same procedure as for ridge regression, train a lasso regression where the regularisation parameter $\lambda$ is selected by cross-validation. 

Compare the estimated coefficients for your lasso to those from the ridge regression -- did lasso result in a more sparse model? (i.e. did it set some coefficients to zero and thus eliminate features from the model)